# 바벨 끝단 감지 모델 학습

YOLOv8을 사용하여 바벨 플레이트(옆면/끝단) 감지 모델을 학습합니다.

## 요구사항
- Google Colab (무료 GPU)
- Roboflow 계정 (데이터셋 다운로드용)

## 1. 환경 설정

In [ ]:
# Ultralytics YOLOv8 설치
!pip install ultralytics roboflow

from ultralytics import YOLO
from roboflow import Roboflow
import os

## 2. Roboflow에서 데이터셋 다운로드

Roboflow Universe에서 바벨 데이터셋을 다운로드합니다.

추천 데이터셋:
- https://universe.roboflow.com/gym-pal/barbell-object-detection
- https://universe.roboflow.com/yolo-project-c2bfs/barbells-detector

In [ ]:
# Roboflow API 키 입력 (https://app.roboflow.com/settings/api 에서 확인)
ROBOFLOW_API_KEY = "YOUR_API_KEY_HERE"

rf = Roboflow(api_key=ROBOFLOW_API_KEY)

# 바벨 데이터셋 다운로드 (workspace/project 이름은 실제 값으로 변경)
project = rf.workspace("gym-pal").project("barbell-object-detection")
dataset = project.version(1).download("yolov8")

## 3. 커스텀 데이터 추가 (선택사항)

바벨 옆면/끝단 이미지를 직접 촬영하여 추가하면 정확도가 높아집니다.

In [ ]:
# Google Drive 마운트 (커스텀 이미지 업로드용)
from google.colab import drive
drive.mount('/content/drive')

# 커스텀 이미지 경로 (Drive에 업로드한 경우)
# custom_images_path = '/content/drive/MyDrive/barbell_images/'

## 4. YOLOv8 모델 학습

In [ ]:
# YOLOv8 nano 모델 로드 (모바일용으로 가벼움)
model = YOLO('yolov8n.pt')

# 학습 실행
results = model.train(
    data=f"{dataset.location}/data.yaml",
    epochs=100,           # 에포크 수
    imgsz=320,            # 입력 이미지 크기 (모바일용)
    batch=16,             # 배치 크기
    name='barbell_detector',
    patience=20,          # Early stopping
)

## 5. 모델 평가

In [ ]:
# 검증 데이터로 평가
metrics = model.val()
print(f"mAP50: {metrics.box.map50}")
print(f"mAP50-95: {metrics.box.map}")

## 6. TFLite로 Export (iOS/Android용)

In [ ]:
# 최적의 모델 로드
best_model = YOLO('runs/detect/barbell_detector/weights/best.pt')

# TFLite로 export (float16 양자화 - iOS Metal 가속 지원)
best_model.export(
    format='tflite',
    imgsz=320,
    half=True,  # float16
)

print("TFLite 모델 생성 완료!")
print("파일 위치: runs/detect/barbell_detector/weights/best_float16.tflite")

## 7. CoreML로 Export (iOS 전용 - 더 빠름)

In [ ]:
# CoreML로 export
best_model.export(
    format='coreml',
    imgsz=320,
    half=True,
)

print("CoreML 모델 생성 완료!")
print("파일 위치: runs/detect/barbell_detector/weights/best.mlpackage")

## 8. 모델 다운로드

In [ ]:
# Google Drive에 저장
!cp runs/detect/barbell_detector/weights/best_float16.tflite /content/drive/MyDrive/barbell_detector.tflite
!cp -r runs/detect/barbell_detector/weights/best.mlpackage /content/drive/MyDrive/barbell_detector.mlpackage

print("모델이 Google Drive에 저장되었습니다!")
print("")
print("다음 단계:")
print("1. barbell_detector.tflite 파일을 다운로드")
print("2. barbell_path_camera/barbell_tracking/assets/ml/ 폴더에 복사")
print("3. 앱 다시 빌드")

## 테스트 (선택사항)

In [ ]:
# 테스트 이미지로 추론
from PIL import Image
import matplotlib.pyplot as plt

# 테스트 이미지 경로
test_image = "path/to/test_image.jpg"

# 추론 실행
results = best_model(test_image)

# 결과 시각화
for r in results:
    im_array = r.plot()
    im = Image.fromarray(im_array[..., ::-1])
    plt.imshow(im)
    plt.axis('off')
    plt.show()